<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Применение-алгоритмов-машинного-обучения-для-задачи-банковского-скоринга" data-toc-modified-id="Применение-алгоритмов-машинного-обучения-для-задачи-банковского-скоринга-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Применение алгоритмов машинного обучения для задачи банковского скоринга</a></span></li><li><span><a href="#Сравнить-работу-алгоритмов-классификации:" data-toc-modified-id="Сравнить-работу-алгоритмов-классификации:-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Сравнить работу алгоритмов классификации:</a></span></li><li><span><a href="#Для-анализа-работы-алгоритмов-использовать-метрики:" data-toc-modified-id="Для-анализа-работы-алгоритмов-использовать-метрики:-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Для анализа работы алгоритмов использовать метрики:</a></span></li><li><span><a href="#Для-разбиения-выборки-в-процессе-кросс-валидации-в-процессе-обучения-можно-использовать-KFold:" data-toc-modified-id="Для-разбиения-выборки-в-процессе-кросс-валидации-в-процессе-обучения-можно-использовать-KFold:-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Для разбиения выборки в процессе кросс-валидации в процессе обучения можно использовать KFold:</a></span></li></ul></div>

# Применение алгоритмов машинного обучения для задачи банковского скоринга

Входными данными являются наборы для обучения и для тестирования:
* bank_scoring_train.csv.gz
* bank_scoring_test.csv.gz

Описание полей:
* SeriousDlqin2yrs - Флаг выхода на просрочку через 90 дней
* RevolvingUtilizationOfUnsecuredLines - Общий баланс по кредитным картам и личным кредитным линиям, за исключением недвижимости, и без задолженности по кредитам, таким как автокредиты, деленные на сумму кредитных лимитов
* age - Возраст заёмщика в годах
* NumberOfTime30-59DaysPastDueNotWorse - Сколько раз заемщик просрочил выплату не более, чем на 30-59 дней, за последние 2 года.
* DebtRatio - Ежемесячные платежи по долгам, алименты и прожиточные расходы, делённые на месячный валовый доход
* MonthlyIncome - Ежемесячный доход
* NumberOfOpenCreditLinesAndLoans - Количество открытых кредитов (например, автокредит или закладная, кредитные карты и т.д.)
* NumberOfTimes90DaysLate - Сколько раз заёмщик просрочил выплату на 90 дней и более.
* NumberRealEstateLoansOrLines - Количество ипотечных кредитов и кредитов на недвижимость
* NumberOfTime60-89DaysPastDueNotWorse - Сколько раз заёмщик просрочил выплату не более, чем на 60-89 дней, за последние 2 года.
* NumberOfDependents - Количество иждивенцев в семье, исключая самого заёмщика (супруга, дети и т.д.)

Целевой переменной является SeriousDlqin2yrs

In [83]:
import pandas
import numpy as np

from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from sklearn.tree import DecisionTreeClassifier

In [84]:
test_data = pandas.read_csv('bank_scoring_test.csv', sep='\t')
train_data = pandas.read_csv('bank_scoring_train.csv', sep='\t')

X_train = train_data.iloc[:,1:]
y_train = train_data.iloc[:,0]

X_test = test_data.iloc[:,1:]
y_test = test_data.iloc[:,0]

In [85]:
# test_data.head()
# train_data.head()

In [86]:
test_data.shape, train_data.shape

((24053, 11), (96216, 11))

In [87]:
np.unique(test_data['SeriousDlqin2yrs'], return_counts=True), np.unique(train_data['SeriousDlqin2yrs'], return_counts=True)

((array([0, 1]), array([22407,  1646])),
 (array([0, 1]), array([89592,  6624])))

# Сравнить работу алгоритмов классификации:
* KNeighborsClassifier (например, с числом ближайших соседей n_neighbors = 1, 10, 50, 100)

http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier

* DecisionTreeClassifier (например, с максимальной глубиной max_depth = 1, 10, 100, None)

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

* LogisticRegression

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

# Для анализа работы алгоритмов использовать метрики:
* f1_score

http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

* roc_auc_score

http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html

* accuracy_score

http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

# Для разбиения выборки в процессе кросс-валидации в процессе обучения можно использовать KFold: 

* http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

kNN - это алгоритм обучения без учителя - мы просто "запоминаем" выборку. Поэтому подберем пораметры на трейне и посмотрим какой результат получим на тестовой выборке.

In [88]:
def k_neighbors_rez(kf, X, y, scoring_name):
    scores = list()
    k_range = [1, 10, 50, 100]
    for k in k_range:
        model = KNeighborsClassifier(n_neighbors=k)
        scores.append(cross_val_score(estimator=model, X=X, y=y, cv=kf, scoring=scoring_name))

    return pandas.DataFrame(scores, k_range).mean(axis=1).sort_values(ascending=False)

In [89]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

accuracy = k_neighbors_rez(kf, X_train, y_train, 'accuracy')
# top_accuracy = accuracy.head(1)
f1 = k_neighbors_rez(kf, X_train, y_train, 'f1')
roc_auc = k_neighbors_rez(kf, X_train, y_train, 'roc_auc')

In [90]:
print('accuracy results:')
print(accuracy)
print('f1 result: ', f1.head(1))
print('roc_auc result: ', roc_auc.head(1))

accuracy results:
1      0.943824
50     0.931186
100    0.931165
10     0.931061
dtype: float64
f1 result:  1    0.560139
dtype: float64
roc_auc result:  1    0.747401
dtype: float64


Как видим все метрики показали что количество ближайших соседей нужно брать равное 1.
Теперь посмотрим на метрики на тестовой выборке.

In [91]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_test, y_test)
knn_pred = knn.predict(X_test)

In [92]:
print(accuracy_score(y_test, knn_pred))
print(f1_score(y_test, knn_pred))
print(roc_auc_score(y_test, knn_pred))

0.9999584251444726
0.9996961409905804
0.9996962332928311


In [97]:
def dec_tree_rez(kf, X, y, scoring_name):
    scores = list()
    k_range = [1, 10, 100, None]
    for k in k_range:
        model = DecisionTreeClassifier(max_depth=k, random_state=17)
        scores.append(cross_val_score(estimator=model, X=X, y=y, cv=kf, scoring=scoring_name))

    return pandas.DataFrame(scores, k_range).mean(axis=1).sort_values(ascending=False)

In [98]:
accuracy = dec_tree_rez(kf, X_train, y_train, 'accuracy')
f1 = dec_tree_rez(kf, X_train, y_train, 'f1')
roc_auc = dec_tree_rez(kf, X_train, y_train, 'roc_auc')

In [110]:
print('accuracy results:')
print(accuracy)
print('f1 result: ', f1)
print('roc_auc result: ', roc_auc)

accuracy results:
NaN    0.942671
100    0.942671
10     0.937069
1      0.931155
dtype: float64
f1 result:  NaN    0.593518
100    0.593518
10     0.341919
1      0.000000
dtype: float64
roc_auc result:  10     0.826388
NaN    0.788033
100    0.788033
1      0.648525
dtype: float64


Тут уже не так явно, но возьмем Nan

In [113]:
tree = DecisionTreeClassifier(max_depth=None, random_state=17)
tree.fit(X_test, y_test)
tree_pred = tree.predict(X_test)

In [114]:
print(accuracy_score(y_test, tree_pred))
print(f1_score(y_test, tree_pred))
print(roc_auc_score(y_test, tree_pred))

0.9999584251444726
0.9996961409905804
0.9996962332928311


Результаты в точности совпали с knn, значит мы одинаково разделили выборку

In [96]:
‘accuracy’, ‘f1’, ‘roc_auc’


















SyntaxError: invalid character in identifier (<ipython-input-96-fa7a1905a580>, line 1)